# ಸಪೋರ್ಟ್ ವೆಕ್ಟರ್ ರೆಗ್ರೆಸರ್ ಬಳಸಿ ಕಾಲ ಸರಣಿ ಭವಿಷ್ಯವಾಣಿ


ಈ ನೋಟ್ಬುಕ್‌ನಲ್ಲಿ, ನಾವು ಹೇಗೆ ಮಾಡುವುದು ಎಂದು ತೋರಿಸುತ್ತೇವೆ:

- 2D ಕಾಲ ಸರಣಿ ಡೇಟಾವನ್ನು SVM ರೆಗ್ರೆಸರ್ ಮಾದರಿಗಾಗಿ ತರಬೇತಿಗೆ ಸಿದ್ಧಪಡಿಸುವುದು
- RBF ಕರ್ಣಲ್ ಬಳಸಿ SVR ಅನ್ನು ಜಾರಿಗೆ ತರುವುದು
- ಪ್ಲಾಟ್‌ಗಳು ಮತ್ತು MAPE ಬಳಸಿ ಮಾದರಿಯನ್ನು ಮೌಲ್ಯಮಾಪನ ಮಾಡುವುದು


## ಮೋಡ್ಯೂಲ್‌ಗಳನ್ನು ಆಮದು ಮಾಡಿಕೊಳ್ಳುವುದು


In [1]:
import sys
sys.path.append('../../')

In [2]:
import os
import warnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
import math

from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from common.utils import load_data, mape

## ಡೇಟಾ ಸಿದ್ಧತೆ


### ಡೇಟಾ ಲೋಡ್ ಮಾಡಿ


In [3]:
energy = load_data('../../data')[['load']]
energy.head(5)

,load
2012-01-01 00:00:00,2698.0
2012-01-01 01:00:00,2558.0
2012-01-01 02:00:00,2444.0
2012-01-01 03:00:00,2402.0
2012-01-01 04:00:00,2403.0


### ಡೇಟಾವನ್ನು ಚಿತ್ರಿಸಿ


In [ ]:
energy.plot(y='load', subplots=True, figsize=(15, 8), fontsize=12)
plt.xlabel('timestamp', fontsize=12)
plt.ylabel('load', fontsize=12)
plt.show()

### ತರಬೇತಿ ಮತ್ತು ಪರೀಕ್ಷಾ ಡೇಟಾ ರಚಿಸಿ


In [ ]:
train_start_dt = '2014-11-01 00:00:00'
test_start_dt = '2014-12-30 00:00:00'

In [ ]:
energy[(energy.index < test_start_dt) & (energy.index >= train_start_dt)][['load']].rename(columns={'load':'train'}) \
    .join(energy[test_start_dt:][['load']].rename(columns={'load':'test'}), how='outer') \
    .plot(y=['train', 'test'], figsize=(15, 8), fontsize=12)
plt.xlabel('timestamp', fontsize=12)
plt.ylabel('load', fontsize=12)
plt.show()

### ತರಬೇತಿಗಾಗಿ ಡೇಟಾ ಸಿದ್ಧಪಡಿಸುವುದು


ಈಗ, ನಿಮ್ಮ ಡೇಟಾವನ್ನು ತರಬೇತಿಗಾಗಿ ತಯಾರಿಸಲು ಫಿಲ್ಟರಿಂಗ್ ಮತ್ತು ಸ್ಕೇಲಿಂಗ್ ಮಾಡುವ ಅಗತ್ಯವಿದೆ.


In [ ]:
train = energy.copy()[(energy.index >= train_start_dt) & (energy.index < test_start_dt)][['load']]
test = energy.copy()[energy.index >= test_start_dt][['load']]

print('Training data shape: ', train.shape)
print('Test data shape: ', test.shape)

ಡೇಟಾವನ್ನು (0, 1) ಶ್ರೇಣಿಯಲ್ಲಿ ಮಾಪಿಸಿ.


In [ ]:
scaler = MinMaxScaler()
train['load'] = scaler.fit_transform(train)
train.head(5)

In [ ]:
test['load'] = scaler.transform(test)
test.head(5)

### ಕಾಲಚರಣಗಳೊಂದಿಗೆ ಡೇಟಾ ರಚನೆ


ನಮ್ಮ SVR ಗಾಗಿ, ನಾವು ಇನ್‌ಪುಟ್ ಡೇಟಾವನ್ನು `[batch, timesteps]` ರೂಪಕ್ಕೆ ಪರಿವರ್ತಿಸುತ್ತೇವೆ. ಆದ್ದರಿಂದ, ನಾವು ಇತ್ತೀಚಿನ `train_data` ಮತ್ತು `test_data` ಅನ್ನು ಪುನರ್‌ರೂಪಗೊಳಿಸುತ್ತೇವೆ ಹಾಗಾಗಿ ಒಂದು ಹೊಸ ಆಯಾಮವು timesteps ಅನ್ನು ಸೂಚಿಸುತ್ತದೆ. ನಮ್ಮ ಉದಾಹರಣೆಗೆ, ನಾವು `timesteps = 5` ಅನ್ನು ತೆಗೆದುಕೊಳ್ಳುತ್ತೇವೆ. ಆದ್ದರಿಂದ, ಮಾದರಿಯ ಇನ್‌ಪುಟ್‌ಗಳು ಮೊದಲ 4 timesteps ಗಾಗಿ ಡೇಟಾಗಳಾಗಿದ್ದು, ಔಟ್‌ಪುಟ್ 5ನೇ timestep ಗಾಗಿ ಡೇಟಾಗಿರುತ್ತದೆ.


In [ ]:
# Converting to numpy arrays

train_data = train.values
test_data = test.values

In [ ]:
# Selecting the timesteps

timesteps=None

In [ ]:
# Converting data to 2D tensor

train_data_timesteps=None

In [ ]:
# Converting test data to 2D tensor

test_data_timesteps=None

In [ ]:
x_train, y_train = None
x_test, y_test = None

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

## SVR ಮಾದರಿ ರಚನೆ ಮಾಡುವುದು


In [ ]:
# Create model using RBF kernel

model = None

In [ ]:
# Fit model on training data

### ಮಾದರಿ ಭವಿಷ್ಯವಾಣಿ ಮಾಡಿ


In [ ]:
# Making predictions

y_train_pred = None
y_test_pred = None

## ಮಾದರಿ ಕಾರ್ಯಕ್ಷಮತೆಯನ್ನು ವಿಶ್ಲೇಷಣೆ ಮಾಡುವುದು


In [ ]:
# Scaling the predictions

y_train_pred = scaler.inverse_transform(y_train_pred)
y_test_pred = scaler.inverse_transform(y_test_pred)

In [ ]:
# Scaling the original values

y_train = scaler.inverse_transform(y_train)
y_test = scaler.inverse_transform(y_test)

In [ ]:
# Extract the timesteps for x-axis

train_timestamps = None
test_timestamps = None

In [ ]:
plt.figure(figsize=(25,6))
# plot original output
# plot predicted output
plt.legend(['Actual','Predicted'])
plt.xlabel('Timestamp')
plt.title("Training data prediction")
plt.show()

In [ ]:
print('MAPE for training data: ', mape(y_train_pred, y_train)*100, '%')

In [ ]:
plt.figure(figsize=(10,3))
# plot original output
# plot predicted output
plt.legend(['Actual','Predicted'])
plt.xlabel('Timestamp')
plt.show()

In [ ]:
print('MAPE for testing data: ', mape(y_test_pred, y_test)*100, '%')

## ಸಂಪೂರ್ಣ ಡೇಟಾಸೆಟ್ ಭವಿಷ್ಯವಾಣಿ


In [ ]:
# Extracting load values as numpy array
data = None

# Scaling
data = None

# Transforming to 2D tensor as per model input requirement
data_timesteps=None

# Selecting inputs and outputs from data
X, Y = None, None

In [ ]:
# Make model predictions

# Inverse scale and reshape
Y_pred = None
Y = None

In [ ]:
plt.figure(figsize=(30,8))
# plot original output
# plot predicted output
plt.legend(['Actual','Predicted'])
plt.xlabel('Timestamp')
plt.show()

In [ ]:
print('MAPE: ', mape(Y_pred, Y)*100, '%')

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ಅಸ್ವೀಕರಣ**:  
ಈ ದಸ್ತಾವೇಜು AI ಅನುವಾದ ಸೇವೆ [Co-op Translator](https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ನಿಖರತೆಯಿಗಾಗಿ ಪ್ರಯತ್ನಿಸುತ್ತಿದ್ದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ತಪ್ಪುಗಳು ಇರಬಹುದು ಎಂದು ದಯವಿಟ್ಟು ಗಮನಿಸಿ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿರುವ ಮೂಲ ದಸ್ತಾವೇಜನ್ನು ಅಧಿಕೃತ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಮಹತ್ವದ ಮಾಹಿತಿಗಾಗಿ, ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ತಪ್ಪು ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವಿಕೆ ಅಥವಾ ತಪ್ಪು ವಿವರಣೆಗಳಿಗೆ ನಾವು ಹೊಣೆಗಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
